In [40]:
import numpy as np
import math
from scipy.special import comb
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
# from tqdm import tqdm
plt.style.use(['science', 'no-latex'])

class IsingLattice:

    def __init__(self, temperature, initial_state, size):
        self.size = size
        self.T = temperature
        self.system = self._build_system(initial_state)

    @property
    def sqr_size(self):
        return (self.size, self.size)

    def _build_system(self, initial_state):
        if initial_state == 'r':
            system = np.random.choice([-1, 1], self.sqr_size)
        elif initial_state == 'u':
            system = np.ones(self.sqr_size)
        else:
            raise ValueError(
                "Initial State must be 'r', random, or 'u', uniform"
            )

        return system

    def _bc(self, i):
        if i >= self.size:
            return 0
        if i < 0:
            return self.size - 1
        else:
            return i

    def energy(self, N, M):
        return -2*self.system[N, M]*(
            self.system[self._bc(N - 1), M] + self.system[self._bc(N + 1), M]
            + self.system[N, self._bc(M - 1)] + self.system[N, self._bc(M + 1)]
        )

    @property
    def internal_energy(self):
        e = 0
        E = 0
        E_2 = 0

        for i in range(self.size):
            for j in range(self.size):
                e = self.energy(i, j)
                E += e
                E_2 += e**2

        U = (1./self.size**2)*E
        U_2 = (1./self.size**2)*E_2

        return U, U_2

    @property
    def heat_capacity(self):
        U, U_2 = self.internal_energy
        return (U_2 - U**2)/(self.T)**2
    
    @property
    def entropy(self):
        N = self.size**2
        Np = int((N+np.sum(self.system))/2)
        return math.log(comb(N, Np, exact=True))/N

    @property
    def magnetization(self):
        return np.abs(np.sum(self.system)/self.size**2)


def run(lattice, epochs):
    for epoch in tqdm(range(epochs)):
        # Randomly select a site on the lattice
        N, M = np.random.randint(0, lattice.size, 2)

        # Calculate energy of a flipped spin
        E = -1*lattice.energy(N, M)

        # Apply Metropolis method
        if E <= 0.:
            lattice.system[N, M] *= -1
        elif np.exp(-E/lattice.T) > np.random.rand():
            lattice.system[N, M] *= -1
    return lattice

def main():
    ## Pre-plotting
    fig, ax = plt.subplots(nrows=1, ncols=3, figsize=[9,3], dpi=300)
    fig.tight_layout(rect=[0, 0, 1, 0.95])
    ax = ax.flatten()
    ## Parameters
    temperature = 1.0
    T = [0.1, 2.269, 4]
    initial_state = "r"
    size = 100
    epochs = 500000
    for i in range(len(T)):
        ## Running
        lattice = IsingLattice(temperature=T[i], initial_state=initial_state, size=size)
        lattice = run(lattice, epochs)
        ## Post-plotting
        ax[i].imshow(lattice.system, interpolation='nearest', cmap='jet')
        ax[i].set_xlabel("T = %.3f" %(T[i]), fontsize=15)
        ax[i].tick_params(axis='both', which='major', labelsize=15)
        ax[i].set_xlim([0,100])
        ax[i].set_ylim([0,100])
    
    fig.suptitle("size = %dx%d" %(size, size), fontsize=18)
    fig.savefig("../data/fig/system-T.pdf", bbox_inches='tight', pad_inches=0, dpi=300)
    fig.savefig("../data/fig/system-T.png", bbox_inches='tight', pad_inches=0, dpi=300)
    plt.close()

if __name__ == "__main__":
    main()

  0%|          | 0/500000 [00:00<?, ?it/s]

  0%|          | 0/500000 [00:00<?, ?it/s]

  0%|          | 0/500000 [00:00<?, ?it/s]